In [ ]:
#1  TSAK DEFINITION
#This project is used to recommend movies for a certain user based on his/her previous rating history.
#The input is the matrix from dataset 'Movielens', which records 100,000 rating information from 671 users and 9066 movies.
#I hope this project is not too small...

In [ ]:
#2 TOOLS AND INFRASTRUCTURE
#Using Pandas to interprete the dataset and sklearn for some data processing.
#Using python language.

#Pre precessing: 1 Split dataset into 80% and 20% as training set and test set respectively.
#                2 Construct user-movie matrix for predicting ratings.      

In [ ]:
#3  EVALUATION METRIC
#Using training set to train the parameters and use root mean square error to evaluate the model.
#Using Gradient Descent to optimaze the loss function and find the best parameters.

In [ ]:
#4  APPROACH
#1 Different to simple collabrative filtering, this project use left-singualr and right-singular 
#matrixs to replace user-movie and movie-user matrix.

#2 This project tries different ways to calculate similarity and find the best one.
#3 This project further use svd++ algorithm and select model by Gradient descent. 

In [ ]:
#5 BASELINE
#Simple collabrative filtering algorithm gives rmse 1.82.

In [ ]:
#6 Conclusion
#Final result: rmse 0.88

In [41]:
%matplotlib inline 
import os
import requests
import zipfile
import numpy as np
import scipy as sp
import matplotlib.pylab as plt 
import math
import random
from numpy import *
import sys
import os
from pandas import Series,DataFrame
import pandas as pd
from sklearn.model_selection import train_test_split
from __future__ import division
import numpy as np
import scipy as sp
from numpy.random import random
import csv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from numpy.random import normal,random,uniform

In [42]:
data_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
r = requests.get(data_url)
mr_zip_file = 'ml-latest-small.zip'
with open(mr_zip_file, 'wb') as out_file:
    out_file.write(r.content)

In [43]:
with zipfile.ZipFile(mr_zip_file,"r") as zip_ref:
    zip_ref.extractall("data")   

In [44]:
print(os.listdir('./data/ml-latest-small')) 
with  open('./data/ml-latest-small/links.csv', 'r') as f:
    _links = f.readlines()
with  open('./data/ml-latest-small/movies.csv', 'r') as f:
    _movies = f.readlines()
with  open('./data/ml-latest-small/ratings.csv', 'r') as f:
    _ratings = f.readlines()    
with  open('./data/ml-latest-small/tags.csv', 'r') as f:
    _tags = f.readlines()        

['links.csv', 'movies.csv', 'ratings.csv', 'README.txt', 'tags.csv']


In [45]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']  
ratings = pd.read_table('./data/ml-latest-small/ratings.csv', sep=',', header=None, names=rnames,engine = 'python') 
ratings=ratings.drop([0])

In [46]:
data = ratings.filter(regex='user_id|movie_id|rating')

In [47]:
h10 = ratings.head(10)  
print(h10)  

   user_id movie_id rating   timestamp
1        1       31    2.5  1260759144
2        1     1029    3.0  1260759179
3        1     1061    3.0  1260759182
4        1     1129    2.0  1260759185
5        1     1172    4.0  1260759205
6        1     1263    2.0  1260759151
7        1     1287    2.0  1260759187
8        1     1293    2.0  1260759148
9        1     1339    3.5  1260759125
10       1     1343    2.0  1260759131


In [48]:
print (data.describe())

       user_id movie_id  rating
count   100004   100004  100004
unique     671     9066      10
top        547      356     4.0
freq      2391      341   28750


In [49]:
def read_file(fliename):
    rates = []
    with open(fliename,encoding='utf-8') as csvfile:
        content = csv.reader(csvfile)
        for row in content:
            rates.append([row[0],row[1],row[2]])
    return rates[1:]

In [50]:
def getRating(char_array):
    rate_array = []
    for row in char_array:
        rate_array.append([int(row[0]),int(row[1]),float(row[2])])
    return rate_array

In [51]:
def getMovieInfo(char_array):
    movie_Id = []
    movie_title = []
    for row in char_array:
        movie_Id.append([int(row[0])])
        movie_title.append([row[1]])
    return np.array(movie_Id), movie_title

In [52]:
movies = read_file('./data/ml-latest-small/movies.csv')
[movieId , movie_title] =  getMovieInfo(movies)

In [53]:
rateInfo = read_file('./data/ml-latest-small/ratings.csv')
rate = getRating(rateInfo)
rate_train, rate_test =train_test_split(rate,test_size=0.20)

In [54]:
bi={}  
bu={}  
qi={}  
pu={}

In [55]:
k_sp=20
ave=np.mean(np.array(rate_train)[:,2])

In [56]:
train_matrix = np.zeros((len(movieId),rate[-1][0]))
test_matrix = np.zeros((len(movieId),rate[-1][0]))
for rate_row in rate_train:
    row_idx = np.argwhere(movieId == rate_row[1])
    train_matrix[row_idx[0][0]][rate_row[0]-1] = rate_row[2]
    bi.setdefault(row_idx[0][0],0)
    bu.setdefault(rate_row[0]-1,0)
    qi.setdefault(row_idx[0][0],random((k_sp,1))/10*(np.sqrt(k_sp))) 
    pu.setdefault(rate_row[0]-1,random((k_sp,1))/10*(np.sqrt(k_sp))) 
for rate_row in rate_test:
    row_idx = np.argwhere(movieId == rate_row[1])
    test_matrix[row_idx[0][0]][rate_row[0]-1] = rate_row[2]    

In [57]:
train_matrix=np.array(train_matrix.T)
test_matrix=np.array(test_matrix.T)

In [58]:
sim_user = pairwise_distances(train_matrix, metric='cosine')
sim_mov = pairwise_distances(train_matrix.T, metric='cosine')

In [59]:
def predict_by_user(rating_matrix, sim_matrix):
    average_rating = rating_matrix.mean(axis=1)
    rating_substract = (rating_matrix-average_rating[:, np.newaxis])
    a=average_rating[:, np.newaxis]
    b=sim_matrix.dot(rating_substract)
    c=np.array([np.abs(sim_matrix).sum(axis=1)]).T
    prediction = a+b/c    
    return prediction
def predict_by_movie(rating_matrix, sim_matrix):
    average_rating = rating_matrix.mean(axis=1)
    rating_substract = (rating_matrix-average_rating[:, np.newaxis])
    a=np.dot(rating_matrix,sim_matrix)
    b=np.array([np.abs(sim_matrix).sum(axis=1)])
    prediction = a/b    
    return prediction

In [60]:
pred_user = predict_by_user(train_matrix,sim_user)
pred_movie = predict_by_movie(train_matrix,sim_mov)

In [61]:
def r_mean_square_error(prediction,test_data):
    prediction = prediction[test_data.nonzero()]
    test_data = test_data[test_data.nonzero()]
    a=mean_squared_error(prediction,test_data)
    a=sqrt(a)
    return sqrt(a)

In [62]:
#result given by simple collabrative filtering


print(r_mean_square_error(pred_movie,test_matrix))
print(r_mean_square_error(pred_user,test_matrix))

1.8905866338416237
1.8297608263191527


In [63]:
p=0.8
k=20

In [64]:
U,sigma,VT=np.linalg.svd(train_matrix)

In [65]:
sigma_matrix=np.eye(k)*sigma[0:k]

In [66]:
matrix_t_opt_m=train_matrix.T.dot(U[:,:k]).dot(sigma_matrix)
matrix_t_opt_u=train_matrix.dot(VT[:,:k]).dot(sigma_matrix)

In [67]:
sim_mov_opt = pairwise_distances(matrix_t_opt_m, metric='cosine')
sim_user_opt = pairwise_distances(matrix_t_opt_u, metric='cosine')

In [68]:
pred_user_opt = predict_by_user(train_matrix,sim_user_opt)
pred_movie_opt = predict_by_movie(train_matrix,sim_mov_opt)

In [69]:
#result given by left/right singular matrix with k=20


print(r_mean_square_error(pred_movie_opt,test_matrix))
print(r_mean_square_error(pred_user_opt,test_matrix))

1.9044610563340694
1.8416340340171184


In [70]:
#find the best k by setting threshold

sigma_threshold=np.sum(sigma**2)*p
k_opt=0
b=0
for i in sigma:
    if b<sigma_threshold:
        b+=i**2
        k_opt+=1        

In [71]:
#result given by left/right singular matrix with k_opt


sigma_matrix_kopt=np.eye(k_opt)*sigma[0:k_opt]
matrix_t_opt_m_k=train_matrix.T.dot(U[:,:k_opt]).dot(sigma_matrix_kopt)
matrix_t_opt_u_k=train_matrix.dot(VT[:,:k_opt]).dot(sigma_matrix_kopt)
sim_mov_opt_k = pairwise_distances(matrix_t_opt_m_k, metric='cosine')
sim_user_opt_k = pairwise_distances(matrix_t_opt_u_k, metric='cosine')
pred_user_opt_k = predict_by_user(train_matrix,sim_user_opt_k)
pred_movie_opt_k = predict_by_movie(train_matrix,sim_mov_opt_k)
print(r_mean_square_error(pred_movie_opt_k,test_matrix))
print(r_mean_square_error(pred_user_opt_k,test_matrix))

1.9034288937833506
1.8363812535779072


In [72]:
def pearson(a,b):
    return 0.5+0.5*corrcoef(a.T,b.T,rowvar=0)[0][1]

In [73]:
sim_user_pears=np.zeros((rate[-1][0],rate[-1][0]))
for i in range(rate[-1][0]):   
    for j in range(rate[-1][0]):
        sim_user_pears[i,j]=pearson(matrix_t_opt_u_k[i,:],matrix_t_opt_u_k[j,:])  

In [74]:
#result given by left/right singular matrix with k_opt and pearson similarity


pred_user_pears = predict_by_user(train_matrix,sim_user_pears)
print(r_mean_square_error(pred_user_pears,test_matrix))

1.8202229817593347


In [75]:
#result given by left/right singular matrix with k_opt and euclidean similarity


sim_mov_opt_k = pairwise_distances(matrix_t_opt_m_k, metric='euclidean')
sim_user_opt_k = pairwise_distances(matrix_t_opt_u_k, metric='euclidean')
pred_user_opt_k = predict_by_user(train_matrix,sim_user_opt_k)
pred_movie_opt_k = predict_by_movie(train_matrix,sim_mov_opt_k)
print(r_mean_square_error(pred_movie_opt_k,test_matrix))
print(r_mean_square_error(pred_user_opt_k,test_matrix))

1.866824912282674
1.8154505812046278


In [76]:
#parameters of svd++


step=25
gamma=0.05
Lambda=0.2

In [77]:
def prediction(i,j):  
        bi.setdefault(j,0)  
        bu.setdefault(i,0)  
        qi.setdefault(j,np.zeros((k_sp,1)))  
        pu.setdefault(i,np.zeros((k_sp,1)))  
        
        result=ave+bi[j]+bu[i]+np.sum(qi[j]*pu[i])  
        if result>5:  
            return 5  
        elif result<1:  
            return 1  
        return result  

In [78]:
#training and gradient descent to optimize the loss function
train_X=np.array(rate_train)
for i in range(step):   
    rmse=0.0  
    _k_=np.random.permutation(train_X.shape[0])  
    for j in range(train_X.shape[0]):  
        i=_k_[j]  
        a=train_X[i][0]  
        b=train_X[i][1]  
        c=train_X[i][2]  
        d=c-prediction(a,b)  
        rmse+=d**2  
        bu[a]+=gamma*(d-Lambda*bu[a])  
        bi[b]+=gamma*(d-Lambda*bi[b])  
        e=qi[b]  
        qi[b]+=gamma*(d*pu[a]-Lambda*qi[b])  
        pu[a]+=gamma*(d*e-Lambda*pu[a])  
    
    result=np.sqrt(rmse/train_X.shape[0])
    print ("the rmse of this step on train data is ",result ) 

the rmse of this step on train data is  0.957579251297
the rmse of this step on train data is  0.88744176378
the rmse of this step on train data is  0.86908270169
the rmse of this step on train data is  0.857565245171
the rmse of this step on train data is  0.849004720374
the rmse of this step on train data is  0.842053590008
the rmse of this step on train data is  0.83629766271
the rmse of this step on train data is  0.830726166256
the rmse of this step on train data is  0.824840970963
the rmse of this step on train data is  0.81975180991
the rmse of this step on train data is  0.814991296573
the rmse of this step on train data is  0.809437270662
the rmse of this step on train data is  0.805940017192
the rmse of this step on train data is  0.801144814198
the rmse of this step on train data is  0.796730372359
the rmse of this step on train data is  0.792547929895
the rmse of this step on train data is  0.787808073958
the rmse of this step on train data is  0.784229127368
the rmse of th

In [79]:
#final result given by svd++ algorithm

_sum=0  
test_X=np.array(rate_test)  
for i in range(test_X.shape[0]): 
    a=test_X[i][0]
    b=test_X[i][1]
    c=test_X[i][2]
    rst=prediction(a,b)    
    _sum+=(rst-c)**2  
    result=np.sqrt(_sum/test_X.shape[0])  
print (result)

0.88667707898
